### Med RAG

In [1]:
import json
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from langchain.vectorstores import FAISS
from openai import OpenAI

d:\AI, ML, and Python\RAG-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_articles(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data['articles']

articles = load_articles('../data/articles.json')

In [4]:
from langchain.schema import Document

documents = [Document(page_content=article['content'], metadata=article.get('metadata', {})) for article in articles]

In [5]:
hf_model = SentenceTransformer("all-MiniLM-L6-v2")

In [6]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
    )
splits = text_splitter.split_documents(documents)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6668\928975099.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Retriever

In [7]:
vectorstore = FAISS.from_documents(
    documents=splits, 
    embedding=embedding
    )
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

KEY-BERT key word search

In [8]:
from keybert import KeyBERT

keybert_model = KeyBERT()

def extract_keywords_from_text(text, top_n=5):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=top_n)
    return [keyword[0] for keyword in keywords]

def search_using_keywords(user_query, k_candidates=3):
    keywords = extract_keywords_from_text(user_query)
    
    relevant_docs = []
    for doc in splits:
        doc_keywords = extract_keywords_from_text(doc.page_content)
        if any(keyword in doc_keywords for keyword in keywords):
            relevant_docs.append(doc)

    return relevant_docs[:k_candidates]

k_candidates = 3  

Generator

In [9]:
import openai
from openai import OpenAI

class LLMClient:
    def __init__(self, api_key, model_name="meta-llama/Llama-3.2-3B-Instruct"):
        self.client = OpenAI(
            base_url="https://api-inference.huggingface.co/v1/",
            api_key=api_key
        )
        self.model_name = model_name

    def get_response_from_model(self, context, user_query):
        prompt_text = f"""
        Answer the following question based on the context provided:
        
        Context:
        {context}
        
        Question: 
        {user_query}
        """

        messages = [{"role": "user", "content": prompt_text}]
        
        completion = self.client.chat.completions.create(
            model=self.model_name, 
            messages=messages, 
            max_tokens=500
        )

        return completion.choices[0].message.content

In [10]:
api_key = "hf_**********************************"  
llm_client = LLMClient(api_key)

In [11]:
def rag_system(user_query):
    retrieved_docs = search_using_keywords(user_query, k_candidates) 
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    response = llm_client.get_response_from_model(context, user_query)
    return response

In [12]:
user_query = "Why does Peterson believe that opportunities often lie in places others avoid?"
response = rag_system(user_query)

print("Response from RAG:", response)

Response from RAG: The text does not mention a person named Peterson, but rather refers to a young man as the narrator's acquaintance. However, based on the context provided, it can be inferred that the young man believes that opportunities often lie in places others avoid because:

1. He has experienced being near the bottom and feeling resentful, which limited his initial perception of a particular place (in this case, a restaurant).
2. He had to work hard, get along with difficult cooks, be polite with customers, and pay attention constantly in order to do a good job.
3. As he worked diligently, he realized that every place he might find himself in had more potential than he initially thought, and that the circumstances he perceived as dire were not entirely unfounded.

In essence, the young man believes that opportunities lie in places others avoid because he was willing to confront the challenges and view his situation with a fresh perspective, rather than letting his initial perc

In [ ]:
import json

questions = [
    "Why does Jordan Peterson emphasize the importance of preserving social institutions?",
    "How does Peterson explain the balance between order and chaos in society?",
    "What is the significance of respecting creative achievement in the modern world?",
    "How does Peterson relate social institutions to personal stability?",
    "Why does Peterson warn against carelessly criticizing systems we do not fully understand?",
    "What does 'the fog' represent in this rule?",
    "Why is it important to confront and address hidden truths in our lives?",
    "How does avoiding clarity lead to unnecessary suffering?",
    "What strategies does Peterson suggest for facing the 'fog' in one's life?",
    "How can this rule be applied in personal relationships?",
    "What does it mean to 'imagine who you could be'?",
    "How can focusing on a clear goal improve one's life trajectory?",
    "What role does self-improvement play in this rule?",
    "How does Peterson explain the importance of single-minded focus on personal growth?",
    "What advice does Peterson give for overcoming obstacles while pursuing an ideal?",
    "How can identifying abdicated responsibilities lead to personal growth?",
    "Why does Peterson believe that opportunities often lie in places others avoid?",
    "How does taking responsibility for neglected areas create meaning in life?",
    "What role does responsibility play in developing character?",
    "How can this rule be applied in professional settings?",
    "How does Peterson define actions that we 'hate' doing?",
    "Why is it important to align one's actions with personal values?",
    "What are the consequences of consistently engaging in activities we despise?",
    "How can someone recognize when they are doing something they hate?",
    "How does this rule relate to integrity and authenticity?",
    "How can focusing on a higher goal change your life's direction?",
    "What role does discipline play in achieving success according to Peterson?",
    "Why does Peterson argue that suffering is an unavoidable part of life?",
    "What does Peterson mean when he talks about the necessity of facing tragedy?",
    "How does confronting adversity contribute to personal growth?",
    "How does the idea of 'meaning' differ from 'happiness' in Peterson's view?",
    "What does Peterson mean when he talks about 'cleaning your room'?",
    "How can self-care and personal responsibility lead to a better life?",
    "Why does Peterson suggest that we should tell the truth, even if it’s difficult?",
    "What is the role of hierarchy in Peterson's worldview?",
    "How does the idea of 'order' relate to personal discipline and responsibility?",
    "What are the dangers of radical ideologies according to Peterson?",
    "How does Peterson explain the relationship between freedom and responsibility?",
    "What does Peterson say about the role of suffering in the process of individuation?",
    "How do societal structures influence individual freedom and meaning?",
    "What role does gratitude play in achieving a meaningful life?",
    "How can learning from failure contribute to success?",
    "What is Peterson’s view on the role of fathers in child development?",
    "Why does Peterson believe that self-sacrifice is necessary for building strong relationships?",
    "How does the concept of 'standing up straight with your shoulders back' relate to personal empowerment?",
    "What does Peterson say about the importance of confronting one's fears?",
    "How can small, incremental improvements lead to a meaningful life?",
    "What role does the idea of 'narrative' play in Peterson's thinking?",
    "How does Peterson explain the importance of setting boundaries?",
    "What does Peterson suggest about the balance between individualism and collectivism?",
    "How can understanding your past help you shape your future?",
    "Why does Peterson emphasize the importance of aiming at the highest goal?",
    "What does Peterson mean by 'life is tragic' and how should we approach it?",
    "How can we avoid the trap of resentment and bitterness in life?",
    "What role does meaningful work play in the development of a good life?",
    "How does Peterson relate responsibility to freedom and choice?"
]

def process_questions(questions):
    results = []

    for question in questions:
        response = rag_system(question)

        result = {
            "question": question,
            "answer": response
        }
        results.append(result)

    return results

answers = process_questions(questions)

with open("questions&answers.json", "w") as file:
    json.dump(answers, file, ensure_ascii=False, indent=4)


In [14]:
for answer in answers[:10]:
    print(f"Question: {answer['question']}\nAnswer: {answer['answer']}\n")

Question: What does 'the fog' represent in this rule?
Answer: In this context, "the fog" is a metaphor that represents uncertainty, misconceptions, or misinformation. It symbolizes the idea of unclear or unclear knowledge that may obscure the truth, making it difficult to distinguish between reality and fantasy.

In the passage, Scarlett's classroom discussion suggests that following rules may not be sufficient to achieve moral behavior, and thatmetarules or understanding the underlying purpose of rules is more important.

Here, "the fog" can be seen as the collective uncertainty or lack of clarity surrounding the established rules or moral principles, which may need to be clarified or revised. Scarlett's advice to seek out the truth and confront uncertain situations is meant to convey the importance of dispelling this "fog" to make informed decisions and take responsible actions.

Question: Why is it important to confront and address hidden truths in our lives?
Answer: According to th

Precision@10

In [15]:
rag_QA = [  
           {
                "question": "Why does Jordan Peterson emphasize the importance of preserving social institutions?",
                "answer": "The context does not mention Jordan, but rather an author who emphasizes the importance of preserving social institutions. The author's reasons for emphasizing the importance of preserving social institutions are as follows:\n\n1. The preservation of social institutions is necessary for maintaining balance between too much order and too much chaos in the world.\n2. Institutions are supported by certain \"ancient mechanisms\" that stabilize and maintain order, which must be repaired when they falter.\n3. The preservation of institutions is also necessary to prevent an existential burden, where the world becomes too chaotic due to the lack of stability and structure.\n4. This balance is preserved through the interaction between social institutions and creative achievement, which suggests that institutions and innovation are intertwined.\n\nThe author's ultimate goal is to maintain stability and dynamism in the world, which is dependent on the preservation and adaptation of social institutions."
            },
            {
                "question": "How does Peterson explain the balance between order and chaos in society?",
                "answer": "The text does not provide an explanation from Jordan Peterson, but rather a description based on the context provided. \n\nAccording to the text, the balance between order and chaos in society is maintained by the hero who embodied the most important of the great forces that make up the human psyche, such as action and perception. This hero serves as a mediator between the twin forces of chaos and order, transforming chaos into habitable order and recasting order into chaos so it can be renewed. The hero's actions are driven by a higher value or end place, which gives society direction and meaning, preventing everything from sinking into meaninglessness and boredom."
            },
            {
                "question": "What is the significance of respecting creative achievement in the modern world?",
                "answer": "The passage suggests that respecting creative achievement is significant in the modern world because it allows for a balance between stability and dynamism. It implies that respecting creativity helps keep the world \"balanced on the narrow line between too much order and too much chaos.\" \n\nIn essence, respecting creative achievement means valuing and supporting the innovative ideas and contributions that shape our world. It requires acknowledging that every creative act, no matter how original or groundbreaking, is likely to transform itself into a useful rule over time. \n\nThis process of respecting creative achievement is essential because it helps fix the perceived \"terrible conundrum\" between order and chaos. By respecting creativity, we can repair and evolve our social institutions, ensuring they remain \"alive and healthy.\" \n\nIn summary, the significance of respecting creative achievement in the modern world is that it helps maintain a delicate balance between stability and innovation, ultimately contributing to the stability and dynamism of our world."
            },
            {
                "question": "How does Peterson relate social institutions to personal stability?",
                "answer": "In the context, relative to social institutions and personal stability, Jordan Peterson relates social institutions to personal stability by stating that our institutions remain \"alive and healthy\" only when our endeavors are \"subsumed under the perfection—the sacredness—of that dual ability.\" \n\nIn other words, the stability and effectiveness of societal institutions are directly tied to how individuals align their personal values and behaviors with those of the institutions. When individuals' personal goals and pursuits are in harmony with the values and expectations of the institutions, they contribute to the overall stability and dynamism of the society. \n\nPeterson suggests that social institutions and personal stability are interconnected and that our personal behavior and values must be in line with the social institutions in order to maintain overall stability."
            },
            {
                "question": "Why does Peterson warn against carelessly criticizing systems we do not fully understand?",
                "answer": "There is no mention of Peterson in the provided context. The text discusses Freud's ideas on self-deception and defense mechanisms, but it does not mention Jordan Peterson or any warnings he may have about critiquing complex systems. Therefore, based on the provided context, it's impossible to answer the question."
            },
            {
                "question": "What does 'the fog' represent in this rule?",
                "answer": "In the context of the provided passage, 'the fog' likely represents the uncertainty, ignorance, or misunderstanding of the rules being referred to. It symbolizes the misty or unclear nature of one's knowledge or comprehension of the rules, allowing for the possibility of either understanding their true necessity and sacredness or not doing so.\n\nIn the rule, the author encourages individuals to \"disperse the fog\" and \"find out what is true\" by exploring and understanding the rules. This suggests that the 'fog' represents the unknown, the unclear, or the potentially deceiving nature of one's current understanding of the rules.\n\nIn essence, 'the fog' is a metaphor for the potential for confusion, ignorance, or misinterpretation of the rules, and the phrase \"disperse the fog\" implies seeking clarity, understanding, and enlightenment to move forward with moral action."
            },
            {
                "question": "Why is it important to confront and address hidden truths in our lives?",
                "answer": "According to the provided context, it is essential to confront and address hidden truths in our lives because taking responsibility can lead to finding a meaningful path, improving our personal lot psychologically, and genuinely making what is intolerably wrong better. It allows us to have our cake and eat it, too, meaning we can address problems while also creating a better life for ourselves."
            },
            {
                "question": "How does avoiding clarity lead to unnecessary suffering?",
                "answer": "According to the context, avoiding clarity leads to unnecessary suffering by creating a \"life murky and foggy; leaves it void, unseen, without form, confused—and leaves you bewildered and astonished.\" This implies that a lack of clarity can cause a lack of understanding and insight, which can lead to poor decision making, mistakes, and a sense of confusion and disorientation, ultimately resulting in unnecessary suffering."
            },
            {
                "question": "What strategies does Peterson suggest for facing the 'fog' in one's life?",
                "answer": "The text does not directly mention the word'strategies' for facing the 'fog' in life, but rather presents characteristics and traits of an individual who chooses to face the 'fog.'\n\nPeterson suggests that one needs to possess the following qualities to face the 'fog':\n\n1. Careful aim: He requires a focused and deliberate approach to addressing the challenges in life.\n2. Wakeful striving and commitment: One needs to be fully engaged and dedicated to overcoming the obstacles in life.\n\nAdditionally, the text emphasizes the importance of 'purest of motivations', the willingness to confront error and change, and the ability to communicate genuinely and honestly."
            },
            {
                "question": "How can this rule be applied in personal relationships?",
                "answer": "Based on the context provided, this rule seems to be about cultivating a sense of peer regulation through maintaining good relationships with colleagues and friends, and employing an emergent ethic in personal relationships. Here's a possible application of this rule in personal relationships:\n\nTo apply this rule, consider the following steps:\n\n1. **Nurture and work on building strong relationships**: As you do with colleagues, strive to create meaningful connections with family and friends. This will help you establish an \"emergent pattern\" of behavior that is accepted and admired by others.\n2. **Understand social norms and expectations**: Pay attention to the unwritten rules and expectations that govern relationships in your social circle. This will help you calculate how your behavior will be perceived over time and by others.\n3. **Strive for consistency and continuity**: Consistency is key to earning respect and trust in personal relationships. By behaving in a responsible and ethical manner across time and situations, you'll establish a pattern that others will respond to positively.\n4. **Aspire to be a \"Great Player\"**: By modeling positive behavior, empathy, and mutual respect, you'll attract like-minded individuals and strengthen your relationships.\n5. **Pay attention to how others respond to you**: Observe how others react to your behavior and adjust your actions accordingly. If someone disapproves or criticism, reflect on why this is the case and make adjustments to improve.\n6. **Prioritize empathy and understanding**: By actively listening and trying to see things from others' perspectives, you'll build stronger connections and foster a sense of trust and cooperation.\n\nBy applying these steps, you can cultivate an emergent ethic in personal relationships, fostering stronger, more meaningful connections with others and contributing to the overall \"well-being of the social group.\""
            }
]

relevant_answers = [
    "Jordan Peterson emphasizes preserving social institutions because they provide stability, meaning, and structure to society. These institutions, such as family, education, and law, help maintain order and safeguard individual well-being, fostering cooperation and shared values that allow society to function effectively.",
    "Peterson explains that order and chaos are complementary forces that need to be balanced. Order provides structure, stability, and predictability, while chaos fosters creativity and growth. A healthy society requires a balance between the two: too much order leads to rigidity, and too much chaos leads to disorder. Both are essential for progress and adaptation.",
    "Peterson highlights that creative achievements push society forward by bringing new ideas and solutions. In the modern world, respecting and fostering creativity is vital for progress, innovation, and addressing complex challenges.",
    "Peterson relates social institutions to personal stability by explaining that they offer structure and meaning. They provide a framework that helps individuals navigate life’s complexities, promoting a sense of security and purpose.",
    "Peterson warns against carelessly criticizing systems we don’t fully understand because such actions can lead to unintended consequences. He argues that social systems, though imperfect, have evolved over time to balance complex needs, and blindly dismantling them can cause chaos and harm.",
    "It means envisioning your highest potential and the best version of yourself, using that vision as a guide to shape your actions and decisions.",
    "Focusing on a clear goal provides direction, motivation, and purpose, which helps channel efforts more effectively and leads to a more fulfilling life trajectory.",
    "Self-improvement is crucial as it helps individuals progress toward their ideal version of themselves, making them better equipped to face life's challenges and reach their goals.",
    "Peterson stresses that relentless focus on personal growth enables you to overcome distractions and build the discipline and resilience needed to transform your life.",
    "Peterson advises breaking down obstacles into manageable steps, confronting them courageously, and staying committed to the ideal, even when progress is slow or difficult."
]

In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(answer, relevant_answer):
    embeddings = hf_model.encode([answer, relevant_answer])
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]


In [31]:
def calculate_precision_at_10(rag_QA, relevant_answers, threshold=0.4):
    precision_at_10 = 0
    total_queries = len(rag_QA)

    for i in range(total_queries):
        question = rag_QA[i]["question"]
        rag_answer = rag_QA[i]["answer"]
        correct_answer = relevant_answers[i] 

        cosine_sim = calculate_cosine_similarity(rag_answer, correct_answer)

        if cosine_sim >= threshold:
            precision_at_10 += 1  

    return precision_at_10 / total_queries

precision_at_10 = calculate_precision_at_10(rag_QA, relevant_answers)

print(f"Precision@10: {precision_at_10:.2f}")

Precision@10: 0.70


Q50, Q90, Q99

In [33]:
import time
import numpy as np

def measure_response_time(query):
    start_time = time.time()
    response = rag_system(query)  
    end_time = time.time()
    return end_time - start_time

queries = [
    "Why does Jordan Peterson emphasize the importance of preserving social institutions?",
    "How does Peterson explain the balance between order and chaos in society?",
    "What is the significance of respecting creative achievement in the modern world?",
    "How does Peterson relate social institutions to personal stability?",
    "Why does Peterson warn against carelessly criticizing systems we do not fully understand?",
    "What does 'the fog' represent in this rule?",
    "Why is it important to confront and address hidden truths in our lives?",
    "How does avoiding clarity lead to unnecessary suffering?",
    "What strategies does Peterson suggest for facing the 'fog' in one's life?",
    "How can this rule be applied in personal relationships?"
]

response_times = []

for query in queries:
    response_time = measure_response_time(query)
    response_times.append(response_time)  

q50 = np.percentile(response_times, 50)
q90 = np.percentile(response_times, 90)
q99 = np.percentile(response_times, 99)

print(f"Precision@10: {precision_at_10:.2f}")
print(f"q50 (median) response time: {q50:.4f} seconds")
print(f"q90 response time: {q90:.4f} seconds")
print(f"q99 response time: {q99:.4f} seconds")

Precision@10: 1.00
q50 (median) response time: 34.1687 seconds
q90 response time: 37.8529 seconds
q99 response time: 37.9644 seconds
